In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv('moviereviews.tsv', sep='\t')

In [3]:
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [4]:
df.shape

(2000, 2)

In [5]:
df.dtypes

label     object
review    object
dtype: object

In [6]:
df.isnull().sum()

label      0
review    35
dtype: int64

In [7]:
df['label'].value_counts()

neg    1000
pos    1000
Name: label, dtype: int64

In [8]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

In [9]:
for col in categorical_columns:
    entirely_empty_strings_count = (df[col] == '').sum()
    space_string_count = df[col].str.isspace().sum()
    print(f'Column {col} has {df[col].isnull().sum()} nulls \
          and {entirely_empty_strings_count} entirely empty strings \
          and {space_string_count} only space strings')
    print('\n')

Column label has 0 nulls           and 0 entirely empty strings           and 0 only space strings


Column review has 35 nulls           and 0 entirely empty strings           and 27 only space strings




In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [12]:
blanks = []

# (index, label, review text)
for i, lb, rv in df.itertuples():
    if rv.isspace():
        blanks.append(i)

In [13]:
blanks

[57,
 71,
 147,
 151,
 283,
 307,
 313,
 323,
 343,
 351,
 427,
 501,
 633,
 675,
 815,
 851,
 977,
 1079,
 1299,
 1455,
 1493,
 1525,
 1531,
 1763,
 1851,
 1905,
 1993]

In [14]:
len(blanks)

27

In [18]:
df.shape

(1938, 2)

In [16]:
# get rid of the empty space rows
df.drop(blanks, inplace = True)

In [17]:
df.shape

(1938, 2)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X = df['review']
y = df['label']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [25]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC())])

In [26]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [27]:
predictions = text_clf.predict(X_test)

In [28]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [29]:
df_cm = pd.DataFrame(confusion_matrix(y_test, predictions), 
                     index = ['pos', 'neg'],
                     columns = ['pos', 'neg'])

In [30]:
df_cm

,pos,neg
pos,235,47
neg,41,259


In [31]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         neg       0.85      0.83      0.84       282
         pos       0.85      0.86      0.85       300

    accuracy                           0.85       582
   macro avg       0.85      0.85      0.85       582
weighted avg       0.85      0.85      0.85       582



In [32]:
print(accuracy_score(y_test, predictions))

0.8487972508591065
